In [1]:
import sys, os, _pickle as pickle
import tensorflow as tf
import numpy as np
import nltk
from sklearn.metrics import f1_score
import keras

Using TensorFlow backend.


In [2]:
data_dir = '../data'                        # Directory for Data and Other files
ckpt_dir = '../checkpoint'                  # Directory for Checkpoints 
word_embd_dir = '../checkpoint/word_embd'   # Directory for Checkpoints of Word Embedding Layer
model_dir = '../checkpoint/modelv1'         # Directory for Checkpoints of Model

In [3]:
word_embd_dim = 100       # Dimension of embedding layer for words
pos_embd_dim = 25         # Dimension of embedding layer for POS Tags
dep_embd_dim = 25         # Dimension of embedding layer for Dependency Types

word_vocab_size = 400001  # Vocab size for Words
pos_vocab_size = 10       # Vocab size for POS Tags
dep_vocab_size = 21       # Vocab size for Dependency Types
word_state_size = 100
other_state_size = 100
relation_classes = 19     # No. of Relation Classes
state_size = 100          # Dimension of States of LSTM-RNNs
batch_size = 10           # Batch Size for training

channels = 3              # No. of types of features to feed in LSTM-RNN
lambda_l2 = 0.0001
max_len_path = 10         # Maximum length of sequence

In [4]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
grap = tf.Graph()

In [5]:
with grap.as_default():
    with tf.name_scope("input"):
    
    # Length of the sequence
        path_length = tf.placeholder(tf.int32, shape=[2, batch_size], name="path1_length") 

        # Words in the sequence
        word_ids = tf.placeholder(tf.int32, shape=[2, batch_size, max_len_path], name="word_ids") 

         # POS Tags in the sequence
        pos_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="pos_ids") 

        # Dependency Types in the sequence
        dep_ids = tf.placeholder(tf.int32, [2, batch_size, max_len_path], name="dep_ids") 

         # True Relation btw the entities
        y = tf.placeholder(tf.int32, [batch_size], name="y")   

In [6]:
word_ids, dep_ids

(<tf.Tensor 'input/word_ids:0' shape=(2, 10, 10) dtype=int32>,
 <tf.Tensor 'input/dep_ids:0' shape=(2, 10, 10) dtype=int32>)

In [8]:
# Embedding Layer of Words 
with grap.as_default():
    with tf.name_scope("word_embedding"):
        W = tf.Variable(tf.constant(0.0, shape=[word_vocab_size, word_embd_dim]), name="W")
        embedding_placeholder = tf.placeholder(tf.float32,[word_vocab_size, word_embd_dim])
        embedding_init = W.assign(embedding_placeholder)
        embedded_word = tf.nn.embedding_lookup(W, word_ids)
        word_embedding_saver = tf.train.Saver({"word_embedding/W": W})

    # Embedding Layer of POS Tags 
    with tf.name_scope("pos_embedding"):
        W = tf.Variable(tf.random_uniform([pos_vocab_size, pos_embd_dim]), name="W")
        embedded_pos = tf.nn.embedding_lookup(W, pos_ids)
        pos_embedding_saver = tf.train.Saver({"pos_embedding/W": W})

    # Embedding Layer of Dependency Types 
    with tf.name_scope("dep_embedding"):
        W = tf.Variable(tf.random_uniform([dep_vocab_size, dep_embd_dim]), name="W")
        embedded_dep = tf.nn.embedding_lookup(W, dep_ids)
        dep_embedding_saver = tf.train.Saver({"dep_embedding/W": W})

In [9]:
embedded_word, W, embedded_dep

(<tf.Tensor 'word_embedding/embedding_lookup/Identity:0' shape=(2, 10, 10, 100) dtype=float32>,
 <tf.Variable 'dep_embedding/W:0' shape=(21, 25) dtype=float32_ref>,
 <tf.Tensor 'dep_embedding/embedding_lookup/Identity:0' shape=(2, 10, 10, 25) dtype=float32>)

In [10]:
# type(embedded_word)
# with tf.Session() as ss:
#     print(embedded_word.eval())
# tf.Print(embedded_word)
type(embedded_word)

tensorflow.python.framework.ops.Tensor

In [ ]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [11]:
# import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v1.version' from 'C:\\Users\\sadam\\Anaconda3\\lib\\site-packages\\tensorflow\\_api\\v1\\version\\__init__.py'>


In [ ]:
tf.reset_default_graph()
# tf.compat.v1.disable_eager_execution()

In [ ]:
# sz_sequence = 100
# n_units = 120
# n_layers = 3
# with tf.variable_scope("word_lstm1"):
#     inputs = tf.keras.layers.Input(batch_shape=(batch_size, sz_sequence, channels))
#     cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
#     state_series = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
#     state_series_word1 = tf.reduce_max(state_series, axis=1)
    
    
#     cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=init_states[0])
#     state_series_word1 = tf.reduce_max(state_series, axis=1)


In [ ]:
# create a BasicRNNCell
# rnn_cell = tf.compat.v1.nn.rnn_cell.BasicRNNCell(state_size)

# # 'outputs' is a tensor of shape [batch_size, max_time, cell_state_size]

# # defining initial state
# initial_state = rnn_cell.zero_state(batch_size, dtype=tf.float32)

# # 'state' is a tensor of shape [batch_size, cell_state_size]
# outputs, state = tf.compat.v1.nn.dynamic_rnn(rnn_cell, embedded_word[0],
#                                    initial_state=initial_state,
#                                    dtype=None)

### Creating LSTM Layers for Different Embeddings

In [ ]:
# sz_sequence = 10
# n_units = 100
# n_layers = 3
# tf.reset_default_graph()
# #  For Word Embeddings
# with tf.variable_scope("word_lstm1"):
#     inputs = tf.keras.layers.Input(batch_shape=(batch_size, sz_sequence, channels))
#     cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
#     state_series = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
#     state_series_word1 = tf.reduce_max(state_series, axis=1)

In [ ]:
state_series

In [12]:
word_sz_sequence = 100
n_units = 100
n_layers = 3
tf.reset_default_graph()

# with grap.as_default():
#  For Word Embeddings
with grap.as_default():
    with tf.variable_scope("word_lstm1"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, word_sz_sequence, channels))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_w1 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
        state_series_word1 = tf.reduce_max(state_series_w1, axis=1)
        print("state_series_w1", state_series_w1)
        print("state_series_word1", state_series_word1)
    #  For Word Embeddings 2
    with tf.variable_scope("word_lstm2"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, word_sz_sequence, channels))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_w2 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
        state_series_word2 = tf.reduce_max(state_series_w1, axis=1)
        print("state_series_w2", state_series_w2)
        print("state_series_word2", state_series_word2)
    

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
state_series_w1 Tensor("word_lstm1/rnn/transpose_1:0", shape=(10, 100, 100), dtype=float32)
state_series_word1 Tensor("word_lstm1/Max:0", shape=(10, 100), dtype=float32)
state_series_w2 Tensor("word_lstm2/rnn_1/transpose_1:0", shape=(10, 100, 100), dtype=float32)
state_series_word2 Tensor("word_lstm2/Max:0", shape=(10, 100), dtype=float32)


In [13]:
#  For POS Embeddings 1
pos_sz_sequence = 25
with grap.as_default():
    with tf.variable_scope("pos_lstm1"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, pos_sz_sequence, channels))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_p1 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
        state_series_pos1 = tf.reduce_max(state_series_p1, axis=1)
        print("state_series_p1", state_series_p1)
        print("state_series_pos1", state_series_pos1)

    #  For POS Embeddings 2
    with tf.variable_scope("pos_lstm2"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, pos_sz_sequence, channels))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_p2 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
        state_series_pos2 = tf.reduce_max(state_series_p2, axis=1)
        print("state_series_w2", state_series_w2)
        print("state_series_pos2", state_series_pos2)

state_series_p1 Tensor("pos_lstm1/rnn_2/transpose_1:0", shape=(10, 25, 100), dtype=float32)
state_series_pos1 Tensor("pos_lstm1/Max:0", shape=(10, 100), dtype=float32)
state_series_w2 Tensor("word_lstm2/rnn_1/transpose_1:0", shape=(10, 100, 100), dtype=float32)
state_series_pos2 Tensor("pos_lstm2/Max:0", shape=(10, 100), dtype=float32)


In [14]:
#  For POS Embeddings 2
dep_sz_sequence = 25
with grap.as_default():
    with tf.variable_scope("dep_lstm1"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, dep_sz_sequence, channels))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_d1 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
        state_series_dep1 = tf.reduce_max(state_series_d1, axis=1)
        print("state_series_d1", state_series_d1)
        print("state_series_dep1", state_series_dep1)

    with tf.variable_scope("dep_lstm2"):
        inputs = tf.keras.layers.Input(batch_shape=(batch_size, dep_sz_sequence, channels))
        cells = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
        state_series_d2 = tf.keras.layers.RNN(cells, stateful=True, return_sequences=True, return_state=False)(inputs)
        state_series_dep2 = tf.reduce_max(state_series_d2, axis=1)
        print("state_series_d2", state_series_d2)
        print("state_series_dep2", state_series_dep2)


state_series_d1 Tensor("dep_lstm1/rnn_4/transpose_1:0", shape=(10, 25, 100), dtype=float32)
state_series_dep1 Tensor("dep_lstm1/Max:0", shape=(10, 100), dtype=float32)
state_series_d2 Tensor("dep_lstm2/rnn_5/transpose_1:0", shape=(10, 25, 100), dtype=float32)
state_series_dep2 Tensor("dep_lstm2/Max:0", shape=(10, 100), dtype=float32)


In [70]:
ce = [tf.keras.layers.GRUCell(n_units) for _ in range(n_layers)]
ce[0]

In [15]:
with grap.as_default():
    state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
    state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

    state_series = tf.concat([state_series1, state_series2], 1)
    print("state_series", state_series)

state_series Tensor("concat_2:0", shape=(10, 600), dtype=float32)


### Deprecated RNN-LSTM Code

In [ ]:
# hidden_states = tf.zeros([channels, batch_size, state_size], name="hidden_state")
# cell_states = tf.zeros([channels, batch_size, state_size], name="cell_state")

# init_states = [tf.contrib.rnn.LSTMStateTuple(hidden_states[i], cell_states[i]) for i in range(channels)]
# tf.reset_default_graph()
# #  For Word Embeddings
# with tf.variable_scope("word_lstm1"):
#     cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[0], sequence_length=path_length[0], initial_state=init_states[0])
#     state_series_word1 = tf.reduce_max(state_series, axis=1)
# #  For Word Embeddings 2
# with tf.variable_scope("word_lstm2"):
#     cell = tf.contrib.rnn.BasicLSTMCell(word_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_word[1], sequence_length=path_length[1], initial_state=init_states[0])
#     state_series_word2 = tf.reduce_max(state_series, axis=1)
# #  For POS Embeddings 1
# with tf.variable_scope("pos_lstm1"):
#     cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[0], sequence_length=path_length[0],initial_state=init_states[1])
#     state_series_pos1 = tf.reduce_max(state_series, axis=1)
# #  For POS Embeddings 2
# with tf.variable_scope("pos_lstm2"):
#     cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_pos[1], sequence_length=path_length[1],initial_state=init_states[1])
#     state_series_pos2 = tf.reduce_max(state_series, axis=1)
# #  For POS Embeddings 2
# with tf.variable_scope("dep_lstm1"):
#     cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[0], sequence_length=path_length[0], initial_state=init_states[2])
#     state_series_dep1 = tf.reduce_max(state_series, axis=1)

# with tf.variable_scope("dep_lstm2"):
#     cell = tf.contrib.rnn.BasicLSTMCell(other_state_size)
#     state_series, current_state = tf.nn.dynamic_rnn(cell, embedded_dep[1], sequence_length=path_length[1], initial_state=init_states[2])
#     state_series_dep2 = tf.reduce_max(state_series, axis=1)

# state_series1 = tf.concat([state_series_word1, state_series_pos1, state_series_dep1], 1)
# state_series2 = tf.concat([state_series_word2, state_series_pos2, state_series_dep2], 1)

# state_series = tf.concat([state_series1, state_series2], 1)

In [16]:
embedded_dep[0], embedded_word
# tf.reset_default_graph()

(<tf.Tensor 'strided_slice:0' shape=(10, 10, 25) dtype=float32>,
 <tf.Tensor 'word_embedding/embedding_lookup/Identity:0' shape=(2, 10, 10, 100) dtype=float32>)

In [17]:
# grap = tf.Graph()
with grap.as_default():
    with tf.name_scope("hidden_layer"):
        W = tf.Variable(tf.truncated_normal([600, 100], -0.1, 0.1), name="W")
        b = tf.Variable(tf.zeros([100]), name="b")
        y_hidden_layer = tf.matmul(state_series, W) + b

    with tf.name_scope("softmax_layer"):
        W = tf.Variable(tf.truncated_normal([100, relation_classes], -0.1, 0.1), name="W")
        b = tf.Variable(tf.zeros([relation_classes]), name="b")
        logits = tf.matmul(y_hidden_layer, W) + b
        predictions = tf.argmax(logits, 1)

print(logits)

In [ ]:
     # True Relation btw the entities
#     y = tf.placeholder(tf.int32, [batch_size], name="y")  

In [18]:
with grap.as_default():
    tv_all = tf.trainable_variables()
    tv_regu = []
    non_reg = ["word_embedding/W:0","pos_embedding/W:0",'dep_embedding/W:0',"global_step:0",'hidden_layer/b:0','softmax_layer/b:0']
    for t in tv_all:
        if t.name not in non_reg:
            if(t.name.find('biases')==-1):
                tv_regu.append(t)

In [19]:
y, logits

(<tf.Tensor 'input/y:0' shape=(10,) dtype=int32>,
 <tf.Tensor 'softmax_layer/add:0' shape=(10, 19) dtype=float32>)

In [20]:
with grap.as_default():
    with tf.name_scope("loss"):
        l2_loss = lambda_l2 * tf.reduce_sum([ tf.nn.l2_loss(v) for v in tv_regu ])
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
        total_loss = loss + l2_loss

    global_step = tf.Variable(0, name="global_step")

    optimizer = tf.train.AdamOptimizer(0.001).minimize(total_loss, global_step=global_step)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
f = open(data_dir + '/vocab.pkl', 'rb')
vocab = pickle.load(f)
f.close()

word2id = dict((w, i) for i,w in enumerate(vocab))
id2word = dict((i, w) for i,w in enumerate(vocab))

unknown_token = "UNKNOWN_TOKEN"
word2id[unknown_token] = word_vocab_size -1
id2word[word_vocab_size-1] = unknown_token

pos_tags_vocab = []
for line in open(data_dir + '/pos_tags.txt'):
        pos_tags_vocab.append(line.strip())

dep_vocab = []
for line in open(data_dir + '/dependency_types.txt'):
    dep_vocab.append(line.strip())

relation_vocab = []
for line in open(data_dir + '/relation_types.txt'):
    relation_vocab.append(line.strip())


rel2id = dict((w, i) for i,w in enumerate(relation_vocab))
id2rel = dict((i, w) for i,w in enumerate(relation_vocab))

pos_tag2id = dict((w, i) for i,w in enumerate(pos_tags_vocab))
id2pos_tag = dict((i, w) for i,w in enumerate(pos_tags_vocab))

dep2id = dict((w, i) for i,w in enumerate(dep_vocab))
id2dep = dict((i, w) for i,w in enumerate(dep_vocab))

pos_tag2id['OTH'] = 9
id2pos_tag[9] = 'OTH'

dep2id['OTH'] = 20
id2dep[20] = 'OTH'

JJ_pos_tags = ['JJ', 'JJR', 'JJS']
NN_pos_tags = ['NN', 'NNS', 'NNP', 'NNPS']
RB_pos_tags = ['RB', 'RBR', 'RBS']
PRP_pos_tags = ['PRP', 'PRP$']
VB_pos_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
_pos_tags = ['CC', 'CD', 'DT', 'IN']

def pos_tag(x):
    if x in JJ_pos_tags:
        return pos_tag2id['JJ']
    if x in NN_pos_tags:
        return pos_tag2id['NN']
    if x in RB_pos_tags:
        return pos_tag2id['RB']
    if x in PRP_pos_tags:
        return pos_tag2id['PRP']
    if x in VB_pos_tags:
        return pos_tag2id['VB']
    if x in _pos_tags:
        return pos_tag2id[x]
    else:
        return 9

In [23]:
f = open(data_dir + '/train_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open(data_dir + '/train_relations.txt'):
#     print(line.strip().split())
    relations.append(line.strip().split()[0])

In [24]:
print(relations)

['Component-Whole(e2,e1)', 'Other', 'Instrument-Agency(e2,e1)', 'Other', 'Member-Collection(e1,e2)', 'Other', 'Cause-Effect(e2,e1)', 'Entity-Destination(e1,e2)', 'Content-Container(e1,e2)', 'Entity-Destination(e1,e2)', 'Member-Collection(e1,e2)', 'Other', 'Message-Topic(e1,e2)', 'Cause-Effect(e2,e1)', 'Instrument-Agency(e2,e1)', 'Message-Topic(e1,e2)', 'Instrument-Agency(e2,e1)', 'Product-Producer(e2,e1)', 'Component-Whole(e2,e1)', 'Member-Collection(e2,e1)', 'Entity-Origin(e1,e2)', 'Member-Collection(e2,e1)', 'Cause-Effect(e1,e2)', 'Other', 'Member-Collection(e2,e1)', 'Other', 'Cause-Effect(e1,e2)', 'Message-Topic(e1,e2)', 'Message-Topic(e1,e2)', 'Component-Whole(e1,e2)', 'Message-Topic(e2,e1)', 'Cause-Effect(e2,e1)', 'Product-Producer(e1,e2)', 'Entity-Destination(e1,e2)', 'Component-Whole(e1,e2)', 'Entity-Origin(e1,e2)', 'Other', 'Component-Whole(e2,e1)', 'Cause-Effect(e1,e2)', 'Instrument-Agency(e2,e1)']


In [25]:
length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [29]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(defer_build=True)

In [ ]:
# f = open('data/word_embedding', 'rb')
# word_embedding = pickle.load(f)
# f.close()

# sess.run(embedding_init, feed_dict={embedding_placeholder:word_embedding})
# word_embedding_saver.save(sess, word_embd_dir + '/word_embd')

In [30]:
# model_dir = r"E:\The University of Texas at Dallas\Fall 2020\Natural Language Processing- CS 6320.501\NLP Project\Relation-Classification-using-Bidirectional-LSTM-Tree\Relation-Classification-using-Bidirectional-LSTM\checkpoint\modelv1"
model = tf.train.latest_checkpoint(model_dir)
saver.restore(sess, model)

ValueError: Can't load save_path when it is None.

In [61]:
tf.train.latest_checkpoint(model_dir)

In [34]:
model_dir

'../checkpoint/modelv1'

In [ ]:
# latest_embd = tf.train.latest_checkpoint(word_embd_dir)
# word_embedding_saver.restore(sess, latest_embd)

In [36]:
# grap.version
from keras import backend as K

#Before prediction
K.clear_session()

In [38]:
# tf.convert_to_tensor(word_dict)
# word_dict
grap

In [61]:
# path_dict
# type(y_dict)
# word_dict[0].shape
dep_dict[0].shape

(10, 10)

In [43]:
num_epochs = 1
for i in range(num_epochs):
    for j in range(num_batches):
        path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
        word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
        pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
        dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
        y_dict = rel_ids[j*batch_size:(j+1)*batch_size]
        
#         path_dict = tf.convert_to_tensor(path_dict)
#         word_dict = tf.convert_to_tensor(word_dict)
#         pos_dict = tf.convert_to_tensor(pos_dict)
#         dep_dict = tf.convert_to_tensor(dep_dict)
#         y_dict = tf.convert_to_tensor(y_dict)
        
        feed_dict = {
            path_length:path_dict,
            word_ids:word_dict,
            pos_ids:pos_dict,
            dep_ids:dep_dict,
            y:y_dict}
        with tf.Session(graph=grap) as sess:
            _, loss, step = sess.run([optimizer, total_loss, global_step], feed_dict)
        if step%10==0:
            print("Step:", step, "loss:",loss)
        if step % 1000 == 0:
            saver.save(sess, model_dir + '/model')
            print("Saved Model")

InvalidArgumentError: You must feed a value for placeholder tensor 'pos_lstm1/input_3' with dtype float and shape [10,25,3]
	 [[node pos_lstm1/input_3 (defined at <ipython-input-13-c96a2c1911c4>:5) ]]

Original stack trace for 'pos_lstm1/input_3':
  File "C:\Users\sadam\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\sadam\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\sadam\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\sadam\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\Users\sadam\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "C:\Users\sadam\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\sadam\Anaconda3\li

In [ ]:
# training accuracy
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("training accuracy", accuracy)

In [ ]:
f = open(data_dir + '/test_paths', 'rb')
word_p1, word_p2, dep_p1, dep_p2, pos_p1, pos_p2 = pickle.load(f)
f.close()

relations = []
for line in open(data_dir + '/test_relations.txt'):
    relations.append(line.strip().split()[0])

length = len(word_p1)
num_batches = int(length/batch_size)

for i in range(length):
    for j, word in enumerate(word_p1[i]):
        word = word.lower()
        word_p1[i][j] = word if word in word2id else unknown_token 
    for k, word in enumerate(word_p2[i]):
        word = word.lower()
        word_p2[i][k] = word if word in word2id else unknown_token 
    for l, d in enumerate(dep_p1[i]):
        dep_p1[i][l] = d if d in dep2id else 'OTH'
    for m, d in enumerate(dep_p2[i]):
        dep_p2[i][m] = d if d in dep2id else 'OTH'

word_p1_ids = np.ones([length, max_len_path],dtype=int)
word_p2_ids = np.ones([length, max_len_path],dtype=int)
pos_p1_ids = np.ones([length, max_len_path],dtype=int)
pos_p2_ids = np.ones([length, max_len_path],dtype=int)
dep_p1_ids = np.ones([length, max_len_path],dtype=int)
dep_p2_ids = np.ones([length, max_len_path],dtype=int)
rel_ids = np.array([rel2id[rel] for rel in relations])
path1_len = np.array([len(w) for w in word_p1], dtype=int)
path2_len = np.array([len(w) for w in word_p2])

for i in range(length):
    for j, w in enumerate(word_p1[i]):
        word_p1_ids[i][j] = word2id[w]
    for j, w in enumerate(word_p2[i]):
        word_p2_ids[i][j] = word2id[w]
    for j, w in enumerate(pos_p1[i]):
        pos_p1_ids[i][j] = pos_tag(w)
    for j, w in enumerate(pos_p2[i]):
        pos_p2_ids[i][j] = pos_tag(w)
    for j, w in enumerate(dep_p1[i]):
        dep_p1_ids[i][j] = dep2id[w]
    for j, w in enumerate(dep_p2[i]):
        dep_p2_ids[i][j] = dep2id[w]

In [ ]:
# test 
all_predictions = []
for j in range(num_batches):
    path_dict = [path1_len[j*batch_size:(j+1)*batch_size], path2_len[j*batch_size:(j+1)*batch_size]]
    word_dict = [word_p1_ids[j*batch_size:(j+1)*batch_size], word_p2_ids[j*batch_size:(j+1)*batch_size]]
    pos_dict = [pos_p1_ids[j*batch_size:(j+1)*batch_size], pos_p2_ids[j*batch_size:(j+1)*batch_size]]
    dep_dict = [dep_p1_ids[j*batch_size:(j+1)*batch_size], dep_p2_ids[j*batch_size:(j+1)*batch_size]]
    y_dict = rel_ids[j*batch_size:(j+1)*batch_size]

    feed_dict = {
        path_length:path_dict,
        word_ids:word_dict,
        pos_ids:pos_dict,
        dep_ids:dep_dict,
        y:y_dict}
    batch_predictions = sess.run(predictions, feed_dict)
    all_predictions.append(batch_predictions)

y_pred = []
for i in range(num_batches):
    for pred in all_predictions[i]:
        y_pred.append(pred)

count = 0
for i in range(batch_size*num_batches):
    count += y_pred[i]==rel_ids[i]
accuracy = count/(batch_size*num_batches) * 100

print("test accuracy", accuracy)